In [ ]:
purchasesPath = "/user/s292129/data/exam_07_2020/Purchases.txt"
out1 = "out1_07_2020"
out2 = "out2_07_2020"

In [ ]:
purchasesRDD = sc.textFile(purchasesPath).cache()
# schema: miD, customerID, date, price

In [ ]:
filteredRDD = purchasesRDD.filter(lambda line: line.split(",")[2].startswith('2019'))

In [ ]:
mID_priceRDD = filteredRDD.map(lambda line: (line.split(',')[0], float(line.split(',')[3])))

In [ ]:
mID_incomeRDD = mID_priceRDD.reduceByKey(lambda v1, v2: v1+v2).cache()

In [ ]:
maxIncome = mID_incomeRDD.reduce(lambda income1, income2: max(income1, income2))

In [ ]:
mID_incomeRDD.filter(lambda pair: pair[1] == maxIncome).keys().saveAsTextFile(out1)

# TASK B

In [ ]:
filtered10YearsRDD = purchasesRDD.filter(lambda line: int(line.split(",")[2].split("/")[0]) in range(2010,2020))

In [ ]:
filtered10YearsRDD.collect()

In [ ]:
def returnPair(line):
    fields = line.split(",")
    mID = fields[0]
    year = fields[2].split("/")[0]
    return ((mID, year), 1)

mIDYear_oneRDD = filtered10YearsRDD.map(returnPair)

In [ ]:
mIDYear_sumPerYear = mIDYear_oneRDD.reduceByKey(lambda v1, v2: v1+v2).cache()

In [ ]:
year_sum = mIDYear_sumPerYear.map(lambda pair: (pair[0][1], pair[1]))

In [ ]:
year_maxPurchasesDict = year_sum.reduceByKey(lambda s1, s2: max(s1,s2)).collectAsMap()

In [ ]:
year_maxPurchasesDict

In [ ]:
filteredPerMaxPurchases = mIDYear_sumPerYear.filter(lambda pair: pair[1] == year_maxPurchasesDict[pair[0][1]])

In [ ]:
filteredPerMaxPurchases.map(lambda pair: (pair[0][0], 1))\
.reduceByKey(lambda v1, v2: v1+v2).filter(lambda pair: pair[1] >= 2)\
.keys().collect()

# Task B, SQL solution